In [ ]:
import pandas as pd
from openai import OpenAI
import json

# Load the API key
config_path = f"{path}/config.json" # PATH OF THE API KEY

with open(config_path, 'r', encoding='utf-8') as f:
  config = json.load(f)
  API_KEY = config.get("API_KEY")

# Define the prompt you want to send to the system
SYSTEM_PROMPT = "You are a helpful assistant that summarizes articles."

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=API_KEY)

def generate_response(input_text, system_prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": input_text
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=0.1, # CHANGE THIS PARAMETER IF NECESSARY
        max_completion_tokens=2048, # CHANGE THIS PARAMETER IF NECESSARY
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

def process_json(prompt, input_file, output_file):
    # Read JSON file
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # For each article in JSON, read "body" and generate summary
    for item in data:
        body_text = item.get("body", "")
        if body_text:
            summary = generate_response(f"{prompt} {body_text}", SYSTEM_PROMPT)
            item["summary"] = summary

    # Write updated data to new JSON file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    # Also create an Excel file from the updated data
    df = pd.DataFrame(data)
    df.to_excel("summaries.xlsx", index=False)

process_json("Summarize the following article:", "test_articles.json", "summaries.json")

json -> excel

In [ ]:
import json
import pandas as pd

def json_to_excel(input_json_file, output_excel_file):
    # Load the JSON file
    with open(input_json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Convert JSON data into a pandas DataFrame
    df = pd.DataFrame(data)

    # Save DataFrame to an Excel file
    df.to_excel(output_excel_file, index=False, sheet_name="Articles")


# Specify input and output file paths
input_json_file = "summaries.json"
output_excel_file = "articles.xlsx"

# Convert JSON to Excel
json_to_excel(input_json_file, output_excel_file)
print(f"Data from {input_json_file} has been saved to {output_excel_file}")


Data from summaries.json has been saved to articles.xlsx
